In [12]:
from jupyterlab_h5web import H5Web

In [36]:
import struct
a = struct.pack('I', 0x04034b50)
print(a)

b'PK\x03\x04'


In [55]:
H5Web("jsmap.nxs")
H5Web("SuperScan (HADF).nxs")

<jupyterlab_h5web.widget.H5Web object>

In [49]:
from IPython.display import JSON

In [50]:
JSON("2022-02-18_Metadata_Kuehbach.nsproj")

<IPython.core.display.JSON object>

In [3]:
import yaml
import flatdict as fd
file_name = "2022-02-18_Metadata_Kuehbach.nsproj"
with open(file_name, 'r') as stream:
    yml = fd.FlatDict(yaml.safe_load(stream), delimiter='/')
    for serialized_path in yml.keys():
        print(f"{serialized_path}\t\t\t{yml[serialized_path]}")

version			3
uuid			aaec9e79-2725-45b9-bd8b-1a70e0a3e4f9
project_data_folders			['2022-02-18_Metadata_Kuehbach Data']
modified			2022-08-31T22:29:39.568453
data_groups			[{'type': 'data_group', 'uuid': '14ba4658-3407-49b8-892b-f81b88ee347d', 'title': 'My Data', 'display_item_references': [], 'data_groups': []}]
display_items			[{'type': 'display_item', 'uuid': '0256be77-7f65-401b-a1ba-d31064e37dfb', 'created': '2022-02-18T13:09:47.976613', 'calibration_style_id': 'calibrated', 'display_properties': {}, 'graphics': [], 'display_layers': [{'data_row': 0, 'fill_color': '#1E90FF', 'data_index': 0}], 'display_data_channels': [{'type': 'display_data_channel', 'uuid': '9c48aa2a-c910-4200-a625-0f01a09a3578', 'brightness': 0.0, 'contrast': 1.0, 'adjustments': [], 'sequence_index': 0, 'collection_index': [0, 0, 0], 'slice_center': 0, 'slice_width': 1, 'data_item_reference': 'fc37bb8f-9de2-4cbd-9eaa-cf3fdc22a356', 'modified': '2022-02-18T20:27:44.597242'}], 'modified': '2022-08-31T21:51:34.436866'

1. Load nsproj walk display_items and compute data_alphabet_value check if such files exist
2. For each such file perform the mapping into an own NXevent_data_em instance
   For the H5 read attribute try to figure out what conceptually the item is

Reproducibly recover the uuid management choices from ns

In [9]:
import uuid
def encode(uuid_: uuid.UUID, alphabet: str) -> str:
    result = str()
    uuid_int = uuid_.int
    while uuid_int:
        uuid_int, digit = divmod(uuid_int, len(alphabet))
        result += alphabet[digit]
    return result

In [51]:
data_item_uuid_str = "e4900934-9332-4706-94cc-6fbe989647d9"
data_item_uuid_str = "72f1cda3-dea5-4a1b-bb85-ee34d5009860"
data_item_uuid_str = "8506c02c-8ba8-48ef-b22f-5e2878abe2d9"
data_item_uuid_str = "b753bea8-2526-4770-875d-52a262c3d9c4"
data_item_uuid_str = "7e19bb9e-78a5-4d1c-8bd3-b60e6cebae42"
# data_item_uuid_str = "9c48aa2a-c910-4200-a625-0f01a09a3578"
data_item_uuid_uuid = uuid.UUID(f'{data_item_uuid_str}')
print(f'data_{encode(data_item_uuid_uuid, "ABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890")}')  # 25 character results

data_O4W29EVTX4DPTR33VRYHLI2QH


In [52]:
H5Web("data_7EPPSHNUFKH6F6A4JCR45J03G.h5")

<jupyterlab_h5web.widget.H5Web object>

In [4]:
import h5py
fnm = "data_7EPPSHNUFKH6F6A4JCR45J03G.h5"
# fnm = "data_O4W29EVTX4DPTR33VRYHLI2QH.h5"
# fnm = "data_24ZGOFFSF7NVOP322TBCKF0YF.h5"
h5r = h5py.File(fnm, "r")
metadata = h5r["data"].attrs["properties"]
print(type(metadata))
h5r.close()
# with open("data_7EPPSHNUFKH6F6A4JCR45J03G.h5.metadata.json", "w") as f:
#     f.write(metadata)
import json
print(type(metadata))
json_object = json.loads(metadata)
print(type(json_object))
with open(f"{fnm}.metadata.json", "w") as f:
    json.dump(metadata, f, ensure_ascii=False)

<class 'str'>
<class 'str'>
<class 'dict'>


In [54]:
JSON(json_object)

<IPython.core.display.JSON object>

In [48]:
H5Web(fnm)

<jupyterlab_h5web.widget.H5Web object>

In [5]:
for count, value in enumerate([0], start=1):
    print(count, value)

1 0


In [6]:
new_shape = []
not new_shape

True

In [45]:
import flatdict as fd
import numpy as np
import yaml
import re
import datetime
import pandas as pd
import os

# file://///wsl.localhost/Ubuntu/home/mkuehbach/SPRINT-8/nexus_code_camp_2022/parser-nexus/nexusparser/tools/yaml2nxdl/sprint9_update_nx_em/test_manual_building/nexus_definitions/build/manual/build/html/classes/contributed_definitions/NXem.html#nxem

# https://stackoverflow.com/questions/3663450/remove-substring-only-at-the-end-of-string
def rchop(s, suffix):
    if suffix and s.endswith(suffix):
        return s[:-len(suffix)]
    return s

# import metadata from nionswift json file
wkdir = os.getcwd()
file_names = [
    "MultiAcquire (Dectris ELA) #1.json",
    "MultiAcquire (HADF) #1.json",
    "Recording of SuperScan (BF).json",
    "Recording of SuperScan (HADF).json",
    "RonchiCam1.json",
    "RonchiCam2.json",
    "Spectrum Image EELS.json",
    "Spectrum Image HADF.json",
    "SuperScan (BF).json",
    "SuperScan (HADF).json",
    "SuperScan (MADF).json"
    ]
file_names = ["HAADF_01.json"]
# file_names = [f"{fnm}.metadata.json"]

# first of all create a global namespace of all keywords
nion_path_id = 1
all_uniq_path_to_id = {}
all_uniq_id_to_path = {}

for f in file_names:
    file_name = f"{wkdir}/{f}"
    print(file_name)

    with open(file_name, 'r') as stream:
        yml = fd.FlatDict(yaml.safe_load(stream), delimiter='/')

    for serialized_path in yml.keys():
        print(f"{serialized_path}\t\t\t{yml[serialized_path]}")

/home/mkuehbach/Sprint14/finishing_in_migrated_places/merge-with-flo-apm-em-sp14-p1/pynxtools/pynxtools/pynxtools/HAADF_01.json
version			1
reader_version			1
large_format			False
spatial_calibrations			[{'offset': -4.0, 'scale': 0.015625, 'units': 'nm'}, {'offset': -4.0, 'scale': 0.015625, 'units': 'nm'}]
intensity_calibration/offset			0.0
intensity_calibration/scale			1.0
intensity_calibration/units			
collection_dimension_count			0
datum_dimension_count			2
metadata/instrument/high_tension			60000.0
metadata/instrument/defocus			2.107734855509256e-09
metadata/instrument/ImageScanned/EHT			60000.0
metadata/instrument/ImageScanned/PMTBF_gain			0.0
metadata/instrument/ImageScanned/PMTDF_gain			1933.59375
metadata/instrument/ImageScanned/StageOutA			-0.03729969210692419
metadata/instrument/ImageScanned/StageOutB			0.01738158857406741
metadata/instrument/ImageScanned/StageOutX			-0.0003063858235397869
metadata/instrument/ImageScanned/StageOutY			-0.0001353885807373515
metadata/instrument

In [ ]:
    # nionswift metadata have the subtility that some paths have lists as values
    # which contain again dictionaries of the next deeper layer with metadata
    # these we also would like to flatten
    # nion_path_id = 1
    # ns_uniq_path_to_id = {}
    # ns_uniq_id_to_path = {}
    for path in yml.keys():
        prefix = path
        if isinstance(yml[prefix], list):
            only_dicts = True
            for obj in yml[prefix]:
                if type(obj) is not dict:
                    only_dicts = False
                # all obj have to be tested

            if only_dicts is True:
                for obj in yml[prefix]:
                    flat = fd.FlatDict(obj, delimiter='/')
                    for key, val in flat.items():
                        keyword = prefix + '/' + key
                        value = val
                        if keyword not in all_uniq_path_to_id.keys():
                            all_uniq_path_to_id[keyword] = (nion_path_id, file_name)
                            # ns_uniq_path_to_id[keyword] = (nion_path_id, value)
                            # ns_uniq_id_to_path[nion_path_id] = (keyword, value)
                            nion_path_id += 1
        # else:
        keyword = path
        value = yml[path]
        if keyword not in all_uniq_path_to_id.keys():
            all_uniq_path_to_id[keyword] = (nion_path_id, file_name)
            # ns_uniq_path_to_id[keyword] = (nion_path_id, value)
            # ns_uniq_id_to_path[nion_path_id] = (keyword, value)
            nion_path_id += 1

    # next file

# create a table for pandas to export to excel/odfpy
nion_path_id = []
nion_path_name = []
nion_path_file = []
for key, val in all_uniq_path_to_id.items():
    nion_path_id.append(val[0])
    nion_path_name.append(key)
    nion_path_file.append(val[1])

df_ns = pd.DataFrame(
    {'nion_path_id_nionswift': nion_path_id,
     'flattened_path_in_nionswift.json': nion_path_name})
df_ns = df_ns.sort_values(by=['flattened_path_in_nionswift.json'])
# 'file_name first found': c})

# remove temporary variables # all_uniq_id_to_path
variables = [
    file_name, file_names, flat, key, keyword, nion_path_file,\
    nion_path_id, nion_path_name, nx_em_id_to_path, nx_em_path_id, \
    nx_em_path_name, nx_em_path_to_id, obj, only_dicts, \
    path, prefix, serialized_path, stream, val, value]
for i in variables:
    del i